In [21]:
import os
import cv2
import pickle
import numpy as np

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold


from tensorflow import keras
import tensorflow as tf

from arcface.dataset import build_dataset
from arcface.losses import ArcLoss
from arcface.network import ArcLayer, L2Normalization, hrnet_v2
from arcface.training_supervisor import TrainingSupervisor

import IPython.display as display
from glob import glob
from tqdm import tqdm
from abc import ABC, abstractmethod

# TODO


1. hrnet (Done)
2. resnet ()
3. efficienthrnet
4. efficientnet


### TFRecord

In [2]:
def image_feature(value):
    """Returns a bytes_list from a string / byte."""
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(value=[tf.io.encode_jpeg(value).numpy()])
    )

def bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value.encode()]))


def float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))


def int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def float_feature_list(value):
    """Returns a list of float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

def create_example(image, label):
    feature = {
        "image": image_feature(image),
        "label": bytes_feature(label),
    }
    return tf.train.Example(features=tf.train.Features(feature=feature))

### Backbone Class make (ToDo Make into a separate module)############################
class RecordOperator(ABC):

    def __init__(self, filename):
        # Construct a reader if the user is trying to read the record file.
        self.dataset = None
        self._writer = None
        if tf.io.gfile.exists(filename):
            self.dataset = tf.data.TFRecordDataset(filename)
        else:
            # Construct a writer in case the user want to write something.
            self._writer = tf.io.TFRecordWriter(filename)

        # Set the feature description. This should be provided before trying to
        # parse the record file.
        self.set_feature_description()

    @abstractmethod
    def make_example(self):
        """Returns a tf.train.example from values to be saved."""
        pass

    def write_example(self, tf_example):
        """Create TFRecord example from a data sample."""
        if self._writer is None:
            raise IOError("Record file already exists.")
        else:
            self._writer.write(tf_example.SerializeToString())

    @abstractmethod
    def set_feature_description(self):
        """Set the feature_description to parse TFRecord file."""
        pass

    def parse_dataset(self):
        # Create a dictionary describing the features. This dict should be
        # consistent with the one used while generating the record file.
        if self.dataset is None:
            raise IOError("Dataset file not found.")

        def _parse_function(example_proto):
            # Parse the input tf.Example proto using the dictionary above.
            return tf.io.parse_single_example(example_proto, self.feature_description)

        parsed_dataset = self.dataset.map(_parse_function)
        return parsed_dataset
#########################################################################################

class ImageDataset(RecordOperator):
    """Construct ImageDataset tfrecord files."""

    def make_example(self, image, label):
        """Construct an tf.Example with image data and label.
        Args:
            image_string: encoded image, NOT as numpy array.
            label: the label.
        Returns:
            a tf.Example.
        """
        
        image_string = tf.image.decode_image(image)
        image_shape = image_string.shape
        

        # After getting all the features, time to generate a TensorFlow example.
        feature = {
            'image/height': int64_feature(image_shape[0]),
            'image/width': int64_feature(image_shape[1]),
            'image/depth': int64_feature(image_shape[2]),
            'image/encoded': image_feature(image_string),
            'label': int64_feature(label),
        }

        tf_example = tf.train.Example(
            features=tf.train.Features(feature=feature))

        return tf_example
    
    def set_feature_description(self):
        self.feature_description = {
            "image/height": tf.io.FixedLenFeature([], tf.int64),
            "image/width": tf.io.FixedLenFeature([], tf.int64),
            "image/depth": tf.io.FixedLenFeature([], tf.int64),
            "image/encoded": tf.io.VarLenFeature(tf.float32),
            "label": tf.io.FixedLenFeature([], tf.int64),
        }
#         example = tf.io.parse_single_example(example, feature_description)
#         example["image"] = tf.io.decode_jpeg(example["image"], channels=3)
#         return example


def create_tfrecord(path = 'datasets/sorted_palmvein_roi/*/*.bmp', tf_record='datasets/train.record'):
    
    converter = ImageDataset(tf_record)
    samples = glob(path)
    total_samples_num = len(samples)
    ids = set()
    print("Total records: {}".format(total_samples_num))
    
    for i, image_path in tqdm(enumerate(samples)):
        image = tf.io.read_file(image_path)
        ids.add(image_path.split('/')[-2])
        label = len(ids)
        tf_example = converter.make_example(image, label)
        # Write the example to file.
        converter.write_example(tf_example)
        
    print("All done. Record file is:\n{}".format(tf_record))


In [3]:
create_tfrecord(path= 'datasets/processed/*/*.bmp',  tf_record='datasets/train_processed.record')

Total records: 12000


0it [00:00, ?it/s]2022-02-20 19:42:40.850230: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-02-20 19:42:40.873349: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-02-20 19:42:41.392367: E tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:927] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-02-20 19:42:41.392493: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1650 Ti computeCapability: 7.5
coreClock: 1.485GHz coreCount: 16 deviceMemorySize: 4.00GiB deviceMemoryBandwidth: 178.84GiB/s
2022-02-20 19:42:41.392627: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-02-20 19:42:41.718388: I tensorflow/s

All done. Record file is:
datasets/train_processed.record


# Training

In [4]:
def train(base_model, name = "hrnetv2", train_files = "datasets/train.record", test_files = None, val_files = None, input_shape = (128, 128, 3),
          num_ids = 600, num_examples = 12000, training_dir = os.getcwd(),
          frequency = 1000, softmax = False, adam_alpha=0.001, adam_epsilon=0.001, batch_size = 8, export_only = False,
          override = False, epochs = 50, restore_weights_only=False):

    '''
    # Deep neural network training is complicated. The first thing is making
    # sure you have everything ready for training, like datasets, checkpoints,
    # logs, etc. Modify these paths to suit your needs.

    name:str = # What is the model's name?
    
    train_files:str = # Where are the training files?

    test_files:str = # Where are the testing files?

    val_files:str = # Where are the validation files? Set `None` if no files available.

    input_shape:tuple(int) = # What is the shape of the input image?

    embedding_size:int = # What is the size of the embeddings that represent the faces?

    num_ids:int = # How many identities do you have in the training dataset?

    num_examples:int = # How many examples do you have in the training dataset?

    # That should be sufficient for training. However if you want more
    # customization, please keep going.

    training_dir:str = # Where is the training direcotory for checkpoints and logs?

    regularizer = # Any weight regularization?

    frequency:int = # How often do you want to log and save the model, in steps?

    # All sets. Now it's time to build the model. There are two steps in ArcFace
    # training: 1, training with softmax loss; 2, training with arcloss. This
    # means not only different loss functions but also fragmented models.

    base_model:model = # First model is base model which outputs the face embeddings.
    '''
    
    # Where is the exported model going to be saved?
    export_dir = os.path.join(training_dir, 'exported', name)
    
    # Then build the second model for training.
    if softmax:
        print("Building training model with softmax loss...")
        model = keras.Sequential([keras.Input(input_shape),
                                  base_model,
                                  keras.layers.Dense(num_ids,
                                                     kernel_regularizer=regularizer),
                                  keras.layers.Softmax()],
                                 name="training_model")
        loss_fun = keras.losses.CategoricalCrossentropy()
    else:
        print("Building training model with ARC loss...")
        model = keras.Sequential([keras.Input(input_shape),
                                  base_model,
                                  L2Normalization(),
                                  ArcLayer(num_ids, regularizer)],
                                 name="training_model")
        loss_fun = ArcLoss()

    # Summary the model to find any thing suspicious at early stage.
    model.summary()

    # Construct an optimizer. This optimizer is different from the official
    # implementation which use SGD with momentum.
    optimizer = keras.optimizers.Adam(adam_alpha, amsgrad=True, epsilon=adam_epsilon)

    # Construct training datasets.
    dataset_train = build_dataset(train_files,
                                  batch_size=batch_size,
                                  one_hot_depth=num_ids,
                                  training=True,
                                  buffer_size=4096)

    # Construct dataset for validation. The loss value from this dataset can be
    # used to decide which checkpoint should be preserved.
    if val_files:
        dataset_val = build_dataset(val_files,
                                    batch_size=batch_size,
                                    one_hot_depth=num_ids,
                                    training=False,
                                    buffer_size=4096)
    else:
        dataset_val = None

    # The training adventure is long and full of traps. A training supervisor
    # can help us to ease the pain.
    supervisor = TrainingSupervisor(model,
                                    optimizer,
                                    loss_fun,
                                    dataset_train,
                                    training_dir,
                                    frequency,
                                    "categorical_accuracy",
                                    'max',
                                    name)

    # If training accomplished, save the base model for inference.
    if export_only:
        print("The best model will be exported.")
        supervisor.restore(restore_weights_only, True)
        supervisor.export(base_model, export_dir)
        quit()

    # Restore the latest model if checkpoints are available.
    supervisor.restore(restore_weights_only)

    # Sometimes the training process might go wrong and we would like to resume
    # training from manually selected checkpoint. In this case some training
    # objects should be overridden before training started.
    if override:
        supervisor.override(0, 1)
        print("Training process overridden by user.")

    # Now it is safe to start training.
    supervisor.train(epochs, num_examples // batch_size)

    # Export the model after training.
    supervisor.export(base_model, export_dir)


In [5]:
# First model is bexported/e model which outputs the face embeddings.
input_shape = (128, 128, 3)
embedding_size = 512
regularizer = keras.regularizers.L2(5e-4)
base_model = hrnet_v2(input_shape=input_shape, output_size=embedding_size,
                      width=18, trainable=True,
                      kernel_regularizer=regularizer,
                      name="embedding_model")

train(base_model, name = "processed_soft", train_files = "datasets/train_processed.record", test_files = None, val_files = None, input_shape = (128, 128, 3),
          num_ids = 600, num_examples = 12000, training_dir = 'outputs/',
          frequency = 1000, softmax = True, adam_alpha=0.001, adam_epsilon=0.001, batch_size = 8, export_only = False,
          override = False, epochs = 12, restore_weights_only=False)

Building training model with softmax loss...
Model: "training_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_model (Functional) (None, 512)               8753922   
_________________________________________________________________
dense_1 (Dense)              (None, 600)               307800    
_________________________________________________________________
softmax (Softmax)            (None, 600)               0         
Total params: 9,061,722
Trainable params: 9,034,830
Non-trainable params: 26,892
_________________________________________________________________
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to 

2022-02-20 19:46:19.083249: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-02-20 19:46:19.104115: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2592010000 Hz


Restoring..
Only the model weights will be restored.
Checkpoint restored: None
Resume training from global step: 0, epoch: 1
Current step is: 0

Epoch 1/12


  0%|------------------------------------------------------------------------------------------| 0/1500 [00:00<?, ?it/s]2022-02-20 19:46:40.881991: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-02-20 19:46:42.605464: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
2022-02-20 19:46:48.914700: W tensorflow/stream_executor/gpu/asm_compiler.cc:63] Running ptxas --version returned 256
2022-02-20 19:46:48.999579: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2022-02-20 19:47:06.013298: W tensorflow/core/common_runtime/bfc_allocator.cc:248] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.38GiB with freed_by_count=0. The caller indicates that 

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


...

Training accuracy: 0.8978, mean loss: 1.99
Monitor value improved from 0.8934 to 0.8978.
Best model found and saved: outputs/model_scout/processed_soft/ckpt-52
Checkpoint saved at global step: 18000, to file: outputs/checkpoints/processed_soft/ckpt-53
Training accuracy: 0.8978, mean loss: 1.99
Monitor value not improved: 0.8978, latest: 0.8978.


100%|>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>| 1500/1500 [07:24<00:00,  3.38it/s, loss=1.26, accuracy=0.898]

Checkpoint saved at global step: 18000, to file: outputs/checkpoints/processed_soft/ckpt-54
Training accomplished at epoch 12
Saving model to outputs/exported/processed_soft ...



2022-02-20 21:15:43.494610: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: outputs/exported/processed_soft/assets
Model saved at: outputs/exported/processed_soft


# Load softmax model and train with softmax = False

In [9]:
checkpoint_dir = 'outputs/exported/processed_soft/'
num_ids = 600
regularizer = keras.regularizers.L2(5e-4)

base_model = keras.models.load_model(checkpoint_dir)



train(base_model, name = "processed_arc", train_files = "datasets/train_processed.record", test_files = None, val_files = None, input_shape = (128, 128, 3),
          num_ids = 600, num_examples = 12000, training_dir = 'outputs/',
          frequency = 1000, softmax = False, adam_alpha=0.001, adam_epsilon=0.001, batch_size = 8, export_only = False,
          override = False, epochs = 12, restore_weights_only=False)


Building training model with ARC loss...
Model: "training_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_model (Functional) (None, 512)               8753922   
_________________________________________________________________
l2_normalization (L2Normaliz (None, 512)               0         
_________________________________________________________________
arc_layer (ArcLayer)         (None, 600)               307200    
Total params: 9,061,122
Trainable params: 9,034,230
Non-trainable params: 26,892
_________________________________________________________________
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the 

 67%|>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>-----------------| 1000/1500 [05:17<02:38,  3.15it/s, loss=30.24, accuracy=0.627]

Training accuracy: 0.6265, mean loss: 31.03
Monitor value improved from 0.0000 to 0.6265.
Best model found and saved: outputs/model_scout/processed_arc/ckpt-1
Checkpoint saved at global step: 1000, to file: outputs/checkpoints/processed_arc/ckpt-2


100%|>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>| 1500/1500 [07:41<00:00,  3.82it/s, loss=17.15, accuracy=0.721]

Training accuracy: 0.7206, mean loss: 27.51
Monitor value improved from 0.6265 to 0.7206.
Best model found and saved: outputs/model_scout/processed_arc/ckpt-3


...

100%|>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>| 1500/1500 [07:12<00:00,  3.91it/s, loss=3.93, accuracy=0.971]

Training accuracy: 0.9713, mean loss: 9.15
Monitor value improved from 0.9698 to 0.9713.
Best model found and saved: outputs/model_scout/processed_arc/ckpt-52
Checkpoint saved at global step: 18000, to file: outputs/checkpoints/processed_arc/ckpt-53
Training accuracy: 0.9713, mean loss: 9.15
Monitor value not improved: 0.9713, latest: 0.9713.


100%|>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>| 1500/1500 [07:19<00:00,  3.41it/s, loss=3.93, accuracy=0.971]

Checkpoint saved at global step: 18000, to file: outputs/checkpoints/processed_arc/ckpt-54
Training accomplished at epoch 12
Saving model to outputs/exported/processed_arc ...


INFO:tensorflow:Assets written to: outputs/exported/processed_arc/assets
Model saved at: outputs/exported/processed_arc


In [10]:
base_model = keras.models.load_model('outputs/exported/processed_arc')

In [15]:
samples = glob('datasets/processed/*/*.bmp')
img = cv2.imread(samples[1]).reshape(-1,128,128,3)

# optimizer = keras.optimizers.Adam(0.001, amsgrad=True, epsilon=0.001)
# loss_fun = keras.losses.CategoricalCrossentropy()
# base_model.compile(optimizer, loss_fun)

base_model.predict(img)[0]

array([ 9.05367756e+00, -9.90905881e-01, -2.78063536e+00,  2.08470440e+00,
       -6.89761162e+00, -7.36667871e+00, -3.39260364e+00,  1.34441814e+01,
       -4.82636929e-01,  1.13399992e+01, -6.95179892e+00, -1.19924097e+01,
        7.88302517e+00, -1.41918335e+01, -4.11912346e+00, -1.46957254e+00,
       -7.96988773e+00, -1.25587597e+01, -3.30817413e+00, -6.44409084e+00,
        1.22559929e+01,  5.35371399e+00,  5.92604065e+00,  6.71351433e+00,
       -1.11080599e+00, -2.78792906e+00, -2.90345979e+00,  2.45971823e+00,
       -6.26247358e+00, -1.03179255e+01,  8.88810825e+00, -1.41048241e+01,
        4.53196001e+00,  5.65346956e+00, -5.67095137e+00, -1.42811990e+00,
        2.07321930e+00, -9.27039719e+00,  7.42131090e+00,  1.54729605e+01,
        4.39252949e+00, -3.40530872e+00, -1.70784206e+01, -1.34897213e+01,
       -2.26406813e+00,  1.14141998e+01,  8.65560341e+00,  1.77637482e+00,
        3.90383005e+00, -1.00459528e+01, -7.04687500e+00,  3.19938278e+00,
       -1.33050752e+00,  

# Classifier

In [17]:
def save_pkl(pkl, path = 'model.pkl'):
    with open(path, 'wb') as f:
        pickle.dump(pkl, f)
    print("saved pkl file at:",path)

def load_pkl(path='model.pkl'):
    with open(path, 'rb') as f:
        pkl = pickle.load(f)
    return pkl

def make_embeddings(dataset_path='datasets/sorted_palmvein_roi/', output_path='outputs/', model_dir='outputs/exported/arcface'):
    # Grab the paths to the input images in our dataset
    print("[INFO] quantifying palms...")
    imagePaths = glob(dataset_path+'/*/*.bmp')
#     imagePaths.extend(glob(dataset_path+'/*/*.jpg'))

    # Initialize model
    embedding_model = keras.models.load_model(model_dir)
       
    # Initialize our lists of extracted facial embeddings and corresponding people names
    knownEmbeddings = []
    knownNames = []

    # Initialize the total number of faces processed
    total = 0

    # Loop over the imagePaths
    for (i, imagePath) in tqdm(enumerate(imagePaths)):
        # extract the person name from the image path
        name = imagePath.split(os.path.sep)[-2]

         # load the image
        img = cv2.imread(imagePath).reshape(-1,128,128,3)
        palms_embedding = embedding_model.predict(img)[0]
        # add the name of the person + corresponding face
        # embedding to their respective list
        knownNames.append(name)
        knownEmbeddings.append(palms_embedding)
        total += 1
        
    print(total, " palms embedded")
#     print(set(knownNames))

    # save to output
    data = {"embeddings": knownEmbeddings, "names": knownNames}
    save_pkl(pkl=data, path=output_path+'db.pkl')
    
def make_model(embeddings_path='outputs/db.pkl', output_path='outputs/'):
    # Load the face embeddings
    data = load_pkl(embeddings_path)
    num_classes = len(np.unique(data["names"]))
    y = np.array(data["names"])
    X = np.array(data["embeddings"])
    
    
    # Initialize Softmax training model arguments
    input_shape = X.shape[1]
    cv = KFold(n_splits=5, random_state=1, shuffle=True)

    model =  MLPClassifier(hidden_layer_sizes=(input_shape, 640, 112, 640, num_classes), activation='tanh',max_iter=10000, batch_size='auto', learning_rate='adaptive',
                           validation_fraction=0.0, solver='adam', early_stopping=False ,verbose=0,random_state=1)

    for train_idx, valid_idx in cv.split(X):
        model.fit(X[train_idx], y[train_idx],)
        print(model.score(X[valid_idx], y[valid_idx]), end='\t')
    
    save_pkl(pkl=model, path=output_path+'model.pkl')
    
    return model

In [18]:
make_embeddings(dataset_path='datasets/processed/', output_path='outputs/exported/processed_arc/', model_dir='outputs/exported/processed_arc/')

[INFO] quantifying palms...


12000it [13:35, 14.71it/s]


12000  palms embedded


In [22]:
make_model(embeddings_path='outputs/exported/processed_arc/db.pkl', output_path='outputs/exported/processed_arc/')

0.5154166666666666	0.6154166666666666	0.5870833333333333	0.5979166666666667	0.49	

MLPClassifier(activation='tanh', hidden_layer_sizes=(512, 640, 112, 640, 600),
              learning_rate='adaptive', max_iter=10000, random_state=1,
              validation_fraction=0.0, verbose=0)

In [34]:
from sklearn.model_selection import GroupShuffleSplit

data = load_pkl('outputs/db.pkl')

splitter = GroupShuffleSplit(test_size=.20, n_splits=600, random_state = 7)
split = splitter.split(data, groups=data["names"])
train_inds, test_inds = next(split)

train = df.iloc[train_inds]
test = df.iloc[test_inds]

ValueError: Found input variables with inconsistent numbers of samples: [2, 12000]

In [37]:
set([i.shape for i in data["embeddings"]])

{(512,)}

In [29]:
import numpy as np